In [1]:
import cv2
import glob
from IPython.display import HTML
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import subprocess
import sys
%matplotlib inline

In [2]:
# repeat for all images
anchors = pd.read_csv('points.csv', delimiter=';').set_index('fname')
npts = 5

# index image
f0 = 'images/IMG_1001.jpeg'
z0 = cv2.imread(f0)
h, w, _ = z0.shape
pt0 = anchors.loc[f0].values.reshape((npts, 2))

pts = np.array(np.c_[pt0[:, 0], h - pt0[:, 1]].tolist(), np.int32)
pts_0 = pts.reshape((-1, 1, 2))
zt0 = cv2.polylines(z0, [pts_0], True, (0, 0, 255), thickness=10)
m0 = np.c_[pts_0[:, 0], np.ones(npts)]

z_all = []
zw_all = []

plot = False

files = sorted(glob.glob('images/*.jpeg'))

for f1 in files:

    z1 = cv2.imread(f1)
    z_all.append(z1)
    
    pt1 = anchors.loc[f1].values.reshape((npts, 2))

    pts = np.array(np.c_[pt1[:, 0], h - pt1[:, 1]].tolist(), np.int32)
    pts_1 = pts.reshape((-1, 1, 2))
    #zt1 = cv2.polylines(z1, [pts_1], True, (0, 0, 255), thickness=10)

    # compute transformation
    p0 = pts_0[1:5, 0].astype('float32')
    p1 = pts_1[1:5, 0].astype('float32')

    m = cv2.getPerspectiveTransform(p1, p0)
    zw = cv2.warpPerspective(z1, m, (z0.shape[1], z0.shape[0]))
    zw_all.append(zw)
    
    if plot:
        zz = ((zw.astype('float') + z0.astype('float'))/2).astype('uint8')
        fig, ax = plt.subplots(figsize=(7, 7))
        ax.imshow(zz[:, :, ::-1])
        plt.show()

In [3]:
subprocess.call('rm autumn.gif', shell=True)

scale = 3

for i, z in enumerate(zw_all):
    z = z[200:-500, 500:-200]
    h, w, _ = z.shape
    z = cv2.resize(z, (w // scale, h // scale))
    
    fname = files[i].split('_')[-1].rstrip('.jpeg')
    fname = '%s-%s-2020' % (fname[:2], fname[-2:])
    z = cv2.putText(z, '%s' % fname, (50, 50), cv2.FONT_HERSHEY_SIMPLEX,  
                   1, (255, 255, 255), 2, cv2.LINE_AA)
    
    outfile = 'tmp_%03d.jpeg' % i
    cv2.imwrite(outfile, z)

cmd = 'convert -delay 1x7 `seq -f tmp_%%03g.jpeg 0 1 %d` -coalesce -loop 0 autumn.gif' % i
subprocess.call(cmd, shell=True)
subprocess.call('rm tmp*.jpeg', shell=True)
HTML('<img src="./autumn.gif">')